In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
cd /content/drive/MyDrive/Mia-Github/Colabs/git-pytorch-projects/pytorch-projects/classification/plant-leaf-diseases

/content/drive/MyDrive/Mia-Github/Colabs/git-pytorch-projects/pytorch-projects/classification/plant-leaf-diseases


### Step1. Data 분할 위한 디렉토리 생성 - Train / Validation / Test



#### 별첨) zip파일의 경우

In [9]:
import os
import shutil

import zipfile
from zipfile import ZipFile

In [37]:
#1. 원본 데이터(with 클래스별 하위 디렉토리 포함)가 있는 곳

# original_dataset_dir = './dataset'

with zipfile.ZipFile('./dataset.zip', 'r') as obj:
  class_list = [n.split('/')[0] for n in obj.namelist()]
  class_list = list(set(class_list))
class_list

['Tomato___Target_Spot',
 'Corn___healthy',
 'Pepper,_bell___healthy',
 'Tomato___Leaf_Mold',
 'Peach___Bacterial_spot',
 'Tomato___Spider_mites Two-spotted_spider_mite',
 'Cherry___healthy',
 'Tomato___Late_blight',
 'Grape___Esca_(Black_Measles)',
 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)',
 'Corn___Common_rust',
 'Corn___Cercospora_leaf_spot Gray_leaf_spot',
 'Potato___healthy',
 'Tomato___Septoria_leaf_spot',
 'Potato___Early_blight',
 'Apple___Apple_scab',
 'Corn___Northern_Leaf_Blight',
 'Apple___healthy',
 'Tomato___Early_blight',
 'Apple___Black_rot',
 'Grape___Black_rot',
 'Pepper,_bell___Bacterial_spot',
 'Tomato___healthy',
 'Strawberry___Leaf_scorch',
 'Apple___Cedar_apple_rust',
 'Tomato___Tomato_mosaic_virus',
 'Grape___healthy',
 'Potato___Late_blight',
 'Tomato___Tomato_Yellow_Leaf_Curl_Virus',
 'Tomato___Bacterial_spot',
 'Strawberry___healthy',
 'Peach___healthy',
 'Cherry___Powdery_mildew']

In [ ]:
#1. 원본 데이터(with 클래스별 하위 디렉토리 포함)가 있는 곳
original_dataset_dir = './dataset'
class_list = os.listdir(original_dataset_dir)

In [28]:
#2. train, val, test 폴더 포함할 디렉토리 생성
base_dir = './splitted'
os.mkdir(base_dir)

In [29]:
#3. 하위 train, val, test 폴더 생성
train_dir = os.path.join(base_dir, 'train') #./splitted/train
os.mkdir(train_dir)

val_dir = os.path.join(base_dir, 'val') #./splitted/val
os.mkdir(val_dir)

test_dir = os.path.join(base_dir, 'test') #./splitted/test
os.mkdir(test_dir)

In [30]:
#3. 하위 train, val, test 폴더 내 클래스별 폴더 생성
for clss in class_list:
  os.mkdir(os.path.join(train_dir, clss))
  os.mkdir(os.path.join(val_dir, clss))
  os.mkdir(os.path.join(test_dir, clss))

### Step2. 데이터 분할과 클래스별 데이터 수 확인

In [31]:
import math

In [ ]:
for clss in class_list:
  path = os.path.join(original_dataset_dir, clss) #./dataset/class명
  fnames = os.listdir(path) #path 내 파일들

  train_size = math.floor(len(fnames)*0.6)
  val_size = math.floor(len(fnames)*0.2)  
  test_size = math.floor(len(fnames)*0.2)

  train_fnames = fnames[:train_size]
  for f in train_fnames:
    src = os.path.join(path, f) #./dataset/class명/파일명
    dst = os.path.join(os.path.join(train_dir, clss), f) #./splitted/train/class명/파일명
    shutil.copyfile(src, dst)

  val_fnames = fnames[train_size:train_size+val_size]
  for f in val_fnames:
    src = os.path.join(path, f) #./dataset/class명/파일명
    dst = os.path.join(os.path.join(val_dir, clss), f) #./splitted/val/class명/파일명
    shutil.copyfile(src, dst) 

  test_fnames = fnames[train_size+val_size:]
  for f in test_fnames:
    src = os.path.join(path, f) #./dataset/class명/파일명
    dst = os.path.join(os.path.join(test_dir, clss), f) #./splitted/test/class명/파일명
    shutil.copyfile(src, dst) 

### Step3. 데이터셋 준비 완료, Dataloader 생성

In [ ]:
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

In [ ]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device('cuda' if USE_CUDA else 'cpu')

In [ ]:
BATCH_SIZE = 256
EPOCH = 30

In [ ]:
#transforms.Compose()
#이미지 데이터 전처리 / 텐서 형태로 변환
transform_base = transforms.Compose([transforms.Resize((64,64)), transforms.ToTensor()])

In [ ]:
#ImageFolder
#데이터셋을 불러오는 메소드
#하나의 클래스 = 하나의 폴더일 경우 사용
train_dataset = ImageFolder(root='./splitted/train', transform=transform_base)
val_dataset = ImageFolder(root='./splitted/val', transform=transform_base)

In [ ]:
#DataLoader
#불러온 이미지 데이터를 주어진 조건에 따라 미니 배치 단위로 분리
#shuffle을 이용해 데이터의 순서가 섞여 모델이 학습시, label 정보 순서를 기억하는 것 방지
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

### Step4. 베이스라인 모델 설계

In [ ]:
import torch.nn as nn
import torch.nn.Function as F
import torch.optim as optim

In [ ]:
#nn.Module: 딥러닝 모델과 관련된 기본적인 함수를 포함하는 클래스
class Net(nn.Module):
    
    #딥러닝 모델에서 사용할 모든 Layer 정의
    def __init__(self):
        super(Net, self).__init__()

        self.conv1 = nn.Conv2d(3, 32, 3, padding=1) #입력 채널수, 출력 채널수, 커널 크기
        self.pool = nn.MaxPool2d(2, 2) #커널 크기, stride
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1) 
        self.conv3 = nn.Conv2d(64, 64, 3, padding=1) 

        self.fc1 = nn.Linear(4096, 512) #입력 채널수, 출력 채널수
        self.fc2 = nn.Linear(512, 33) #33은 출력 채널수로 부류 클래스 수와 동일

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool(x)
        x = F.dropout(x, p=0.25, training=self.training) # 25% 노드 dropout, training은 학습 모드 적용 의미

        x = self.conv2(x)
        x = F.relu(x)
        x = F.dropout(x, p=0.25, training=self.training)

        x = self.conv3(x)
        x = F.relu(x)
        x = self.pool(x)
        x = F.dropout(x, p=0.25, training=self.trainig)

        x = x.view(-1, 4096) #Flatten
        x = self.fc1(x)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training) #training은 학습 모드와 검증 모드를 다르게 적용하기 위해 존재
                                                        #즉, 학습 과정에서는 일부 노드를 랜덤 제외하지만
                                                        #    평가 과정에서는 모든 노드 사용하기에
        x = self.fc2(x)

        return F.log_softmax(x, dim=1) #데이터가 각 클래스에 속할 확률을 output으로 추출

model_base = Net().to(DEVICE) #to(DEVICE)를 통해 모델을 현재 사용중인 장비에 할당
optimizer = optim.Adam(model_base.parameters(), lr=0.001)

### Step5. 모델 학습을 위한 함수

In [ ]:
def train(model, train_loader, optimizer):
    model.train() #학습 모드 설정

    for batch_id, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad() #이전 batch의 gradient값이 optimizer에 저장되어 있으므로 초기화
        
        output = model(data)
        loss = F.cross_entropy(output, target) #예측값과 target 사이의 loss 계산, 분류문제에는 cross_entropy 사용
        loss.backward() #역전파를 통해 gradient값을 각 파라미터에 할당

        optimizer.step() #gradient 이용해 모델의 파라미터 값 업데이트

### Step6. 모델 평가를 위한 함수

In [ ]:
def evaluate(model, test_loader):
    model.eval() #평가모드 설정

    test_loss = 0
    correct = 0

    with torch.no_grad(): #모델 평가에서는 모델의 파라미터값을 업데이트하지 않음
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)

            test_loss += F.cross_entropy(output, target, reduction='sum').item()

            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
            #target.view_as(pred): pred 텐서와 동일 모양으로 정렬
            #pred.eq(): 값이 일치하면 1 불일치하면 0 반환


    test_loss /= len(test_loader.dataset) #미니 배치 평균 loss 계산
    test_accuracy = 100.*correct/len(test_loader.dataset) #미니 배치 평균 정확도 계산

    return test_loss, test_accuracy

### Step7. 모델 학습 실행하기

In [1]:
import time
import copy

In [ ]:
def train_baseline(model, train_loader, val_loader, optimizer, num_epochs=30):
  best_acc=0.0
  best_model_wts = copy.deepcopy(model.state_dict())

  for epoch in range(1, num_epochs+1):
    since = time.time()
    train(model, train_loader, optimizer)
    train_loss, train_acc = evaluate(model, train_loader)
    val_loss, val_acc = evaluate(model, val_loader)

    if val_acc > best_acc:
      best_acc = val_acc
      best_model_wts = copy.deepcopy(model.state_dict())
    
    time_elapsed = time.time() - since
    print("--------------------- epoch {} ---------------------".format(epoch))

    print("train Loss: {:.4f}, Accuracy: {:.2f}%".format(train_loss, train_acc))
    print("val Loss: {:.4f}, Accuracy: {:.2f}%".format(val_loss, val_acc))
    print("Completed in {:.0f}m {:.0f}s".format(time_elapsed//60, time_elapsed%60))
  
  model.load_state_dic(best_model_wts)
  return model

base = train_baseline(model_base, train_loader, val_loader, optimizer, EPOCH)

torch.save(base, 'baseline.pt')